In [1]:
import cv2
import numpy as np
from pathlib import Path
from os import listdir
from os.path import isfile, join
import pandas as pd
import json

In [ ]:
def updatePoint(event,x,y,flags,param):
    global POINTS
    if event > 0:
        if event == 1:
            POINTS.append([x,y])
            draw()
        if event == 2:
            if len(POINTS)> 0:
                POINTS.pop(len(POINTS)-1)
                draw()

def draw():
    global SRC
    global POINTS
    image = SRC.copy()
    for point in POINTS:
        x = point[0]
        y = point[1]
        cv2.circle(image,(x, y),5,(255,0,0),-1)
    cv2.imshow(window,image)

def saveSeiling(file):
    global ANSWERS
    global POINTS
    saved = False
    for row in ANSWERS:
        if row['file'] == file:
            saved = True
            row['points'] = POINTS
    if saved == False:
        ANSWERS.append({
            'file': file,
            'points': POINTS
        })
    with open('answers.json', 'w') as json_file:
        json.dump(ANSWERS, json_file)
    
def openImage(file):
    global ANSWERS
    global SRC
    global POINTS
    
    POINTS = []
    for asw in ANSWERS:
        if asw['file'] == file:
            POINTS = asw['points']
            
    path = join('img/', file)
    SRC = cv2.imread(path, 1)
    cv2.setMouseCallback(window, updatePoint)
    draw()
    

In [ ]:
window = 'image'

SRC = np.zeros((300,300,3), np.uint8)
IND = 0
POINTS = []
with open('answers.json') as json_file:
    ANSWERS = json.load(json_file)

IMAGES = [f for f in listdir('img') if isfile(join('img/', f))]
cv2.namedWindow(window)

while True:
    key = cv2.waitKeyEx(0)
    if key == 13: # back
        saveSeiling(IMAGES[IND])
        IND += 1
        if IND > (len(IMAGES)-1):
            IND = 0
    if key == 2424832: # back
        IND -= 1
        if IND < 0:
            IND = len(IMAGES)-1
    if key == 2555904: # forward
        IND += 1
        if IND > (len(IMAGES)-1):
            IND = 0
    if key == 27: break # 'ESC'
    openImage(IMAGES[IND])
    POINTS
cv2.destroyAllWindows()